In [4]:
# declaring all dependancies    
import qutip as qt
import numpy as np
from quantum_logical.pulsesim import QuantumSystem, Pulse
from quantum_logical.pulsesim.mode import QubitMode, SNAILMode, CavityMode
from quantum_logical.pulsesim.build_hamiltonian import Build_hamiltonian

In [37]:
# adding all of the necessary elements to the full creation of the system 
qubit_dim = 2

# frequencies 
w1_un = 4
w2_un = 6
w3_un = 4.6666666666666666
ws_un = 5.6666666666666666

# coupling strength coefficient 
g1s = 2 * np.pi * 0.05467
g2s = 2 * np.pi * 0.0435
g3s = 2 * np.pi * 0.04875

# create array of the coefficients should be useful 
gs = [g1s, g2s, g3s]

# unknown value as of now 
g3 = .4

In [30]:
# creating the qubits and the hilbert space
qubit1 = QubitMode(
    mode_type="Qubit", name="q1", dim=2, freq=w1_un, alpha=-0.161, T1=1e2, T2=5e1
)
qubit2 = QubitMode(
    mode_type="Qubit", name="q2", dim=2, freq=w2_un, alpha=-0.1275, T1=1e2, T2=5e1
)
qubit3 = QubitMode(
    mode_type="Qubit", name="q3", dim=2, freq=w3_un, alpha=-0.160, T1=1e2, T2=5e1
)
qubits = [qubit1, qubit2, qubit3]
snail = SNAILMode(mode_type="Snail", name="s", freq=ws_un, g3=0.3, dim=10, T1=1e3, T2=5e2)
_couplings = {
    frozenset([qubit1, snail]): 2 * np.pi * 0.05467,
    frozenset([qubit2, snail]): 2 * np.pi * 0.0435,
    frozenset([qubit3, snail]): 2 * np.pi * 0.04875
}

qs = QuantumSystem(qubits + [snail], couplings=_couplings)

system = []
system.extend(qubits)
system.extend([snail])


In [35]:
# creating the linear parts of the hamiltonian  
# the qubit and snail separate terms
H_no_interaction = [i.freq * qs.modes_a[i] * qs.modes_a_dag[i] for i in system]


# creating the snail mixing with the qubits terms
H_snail_qubits = [gs[qubits.index(i)] * (qs.modes_a[i] * qs.modes_a_dag[snail] + qs.modes_a[snail] * qs.modes_a_dag[i]) for i in qubits]

In [54]:
# creating the non-linear parts of the hamiltonian 
# adding the snail third order term first 
H_snail_third = g3 * ((qs.modes_a[snail] + qs.modes_a_dag[snail]) ** 3)

H_total_no_time = []
H_total_no_time.append(H_no_interaction)
H_total_no_time.append(H_snail_qubits)
H_total_no_time.append(H_snail_third)

In [58]:
# adding the time dependent terms 
# this might have to be done with a constant pulse first but it should be easy to implement the more complex ones 
pulse = 2

H_time = (qs.modes_a[snail] + qs.modes_a_dag[snail])

H = [H_total_no_time,[pulse, H_time]]


In [59]:
# create the intial state 
psi0 = qt.tensor(qt.basis(qubit_dim,1), qt.basis(qubit_dim,0), qt.basis(qubit_dim,0), qt.basis(snail.dim))
rho0 = psi0 * psi0.dag()

In [60]:
# run mesolve over the hamiltonian 
tlist = np.linspace(0,200,100)
result = qt.mesolve(H= H, rho0 = rho0, tlist = tlist)

ValueError: coefficient format not understood

In [ ]:
# will optimize over the amplitude of the pulse and maybe frequency 